-----
Build a Large Language Model
Sebastian Raschka
-----

# Working with text data

  - During the pretraining stage, LLMs process text, one word at a time.

## 2.1 - Understanding word embeddings

  - Converting data into a vector format is referred to as embedding.  An embedding is a mapping from discrete objects, such as words, images, or even entire documents, to point in a continuous vector space - the primary purpose of embeddings is to convert nonnumeric data into a format that neural networks can process.
  - In addition to word embeddings, there are also embedding for sentences, paragraphs, or whole documents.
  - Retrieval-augmented generation combines generation (like producting text) with retrieval (like searching an external knowledge base) to pull relevant information when generating text.

## 2.2 - Tokenizing text

  -


